## This notebook is a short introduction to keras and TF 2.0

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
/
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

import cProfile

### Basic of TF 2.0

In [2]:
tf.executing_eagerly()

True

### Computing gradients

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks. During eager execution, use tf.GradientTape to trace operations for computing gradients later

In [3]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as tape:
    loss = w * w
grad = tape.gradient(loss,w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


# Train a model

## Use keras

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [5]:
x_train = tf.cast(x_train[..., tf.newaxis] / 255, tf.float32),
x_test = tf.cast(x_test[..., tf.newaxis] / 255, tf.float32),

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

### 第一种方式 

In [8]:
#Build the model using Sequential
mnist_model_f = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, [5, 5], activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64, [5, 5], activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [9]:
mnist_model_f.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               819328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [12]:
mnist_model_f.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3), loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [13]:
mnist_model_f.fit(x_train,y_train, batch_size=128, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 198s 3ms/sample - loss: 2.2681 - accuracy: 0.1875
Epoch 2/3
60000/60000 [==============================] - 192s 3ms/sample - loss: 2.0233 - accuracy: 0.3801
Epoch 3/3
60000/60000 [==============================] - 187s 3ms/sample - loss: 1.3513 - accuracy: 0.5775


In [14]:
score = mnist_model_f.evaluate(x_test, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [15]:
print('Test score: ', score[0])

Test score:  0.7701849462509155


In [16]:
print('Accuracy: %.2f%%' % (score[1] * 100))

Accuracy: 82.58%


### 第二种方式

In [17]:
#Build the model using Model
inputs = tf.keras.Input(shape=(None, None, 1), name="digits")
conv_1 = tf.keras.layers.Conv2D(32, [5, 5], activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(64,[5, 5], activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_s = tf.keras.Model(inputs=inputs, outputs=outputs)

In [18]:
mnist_model_s.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, None, None, 32)    832       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, None, None, 64)    51264     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                650       
Total params: 52,746
Trainable params: 52,746
Non-trainable params: 0
_________________________________________________________________


## Use TF 2.0 

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [20]:
dataset = tf.data.Dataset.from_tensor_slices((tf.cast(x_train[..., tf.newaxis] / 255, tf.float32), tf.cast(y_train,tf.int64)))
dataset = dataset.shuffle(1000).batch(32)

In [21]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_history = []

In [23]:
for epoch in range(5):  
    for (batch, (images, labels)) in enumerate (dataset):  
        with tf.GradientTape() as tape:
            logits = mnist_model(images, training=True)
            loss_value = loss(labels, logits)
            
        grads = tape.gradient(loss_value, mnist_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))
        loss_history.append(loss_value)
    print("Epoch {} finishted".format(epoch))

Epoch 0 finishted
Epoch 1 finishted
Epoch 2 finishted
Epoch 3 finishted
Epoch 4 finishted
